# Importing data

Importing data means taking data in some form, and preparing it so that we can express that data as nodes and edges. On its own, this is not too challenging - it mostly means converting data formats. The harder part is harmonizing the data, so that the fields used across imported databases are consistent enough that we can link consumers and supplier.

Let's make this more concrete with an example. In the file `lci-carbon-fiber.xlsx` we have data from the publication [Ecological assessment of fuel cell electric vehicles with special focus on type IV carbon fiber hydrogen tank](https://www.sciencedirect.com/science/article/abs/pii/S0959652620333229). As this data is from Excel, it is tabular, and so on its surface looks different than the graph:

<img src="images/spreadsheet.png">

However, this difference is mostly cosmetic. Both the _document_ and _graph_ perspectives are showing the same information, but with a different emphasis and organizing structure. In the graph perspective, edges are independent objects with their own metadata, and their sources and targets are given as [pointers](https://en.wikipedia.org/wiki/Pointer_(computer_programming)) to the node objects. In the document perspective, edges are subsumed in the definition of the nodes, and because most input data formats don't have pointers, references to input or output flows are defined by the attributes of thoses flows.

Because we only have flow attributes, we need to define a way that we associate those attributes with nodes in our existing databases. This is trickier than you might think, as those is no guarantee that two data providers will use the same labels for things like locations or units; indeed, sometimes we even find different labels for the same attributes.

Therefore, Brightway treats IO as a classic [ETL pipeline](https://en.wikipedia.org/wiki/Extract,_transform,_load), and applies a series of transformation functions to prepare the data and find the correct flows. Let's look at our real-world example:

In [2]:
import bw2data as bd
import bw2io as bi
from pathlib import Path

The example data is built on top of ecoinvent. You should update the project name to a project with ecoinvent 3.10 already installed.

In [3]:
imp = bi.ExcelImporter(Path.cwd() / "lci-carbon-fiber.xlsx")

Extracted 1 worksheets in 0.02 seconds


Before we make any changes, let's see what the data looks like in its raw form:

In [4]:
imp.data[0]

{'comment': '1 kg of finished carbon fiber Toray T700 G. Carbon fiber manufacturing. Source: based on Ellringmann et al. (2016), Gill et al. (2016), Mützel (2012).',
 'source': 'Benitez A, Wulf C, de Palmenaer A, Lengersdorf M, Röding T, Grube T, et al. Ecological assessment of fuel cell electric vehicles with special focus on type IV carbon fiber hydrogen tank. J Clean Prod 2021;278:123277. https://doi.org/10.1016/j.jclepro.2020.123277.',
 'location': 'RER',
 'production amount': 1,
 'reference product': 'polyacrylonitrile',
 'unit': 'kilogram',
 'name': 'polyacrylonitrile production (PAN) by polymerisation',
 'exchanges': [{'name': 'polyacrylonitrile production (PAN) by polymerisation',
   'amount': 1,
   'database': 'carbon fiber',
   'location': 'RER',
   'unit': 'kilogram',
   'type': 'production',
   'reference product': 'polyacrylonitrile'},
  {'name': 'market for heat, from steam, in chemical industry',
   'amount': 49.71610169491525,
   'database': 'ecoinvent',
   'location': 

This is actually aleady quite close to the final form. In this case we are lucky as the import data was designed to be used in Brightway. Normally we would need to apply transformation functions; lets see what those default transformation functions would be:

In [5]:
imp.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.08 seconds


We can look at the imported data statistics:

In [6]:
imp.statistics()

Graph statistics for `carbon fiber` importer:
10 graph nodes:
	None: 10
109 graph edges:
	technosphere: 90
	production: 10
	biosphere: 9
27 edges to the following databases:
	carbon fiber: 19
	ecoinvent-3.10-biosphere: 8
20 unique unlinked edges (82 total):
	technosphere: 19
	biosphere: 1




(10, 109, 82, 0)

We can iterate over the unlinked edges to get a sense for what we are missing:

In [7]:
for edge, _ in zip(imp.unlinked, range(5)):
    print(edge)

{'name': 'market for heat, from steam, in chemical industry', 'amount': 49.71610169491525, 'database': 'ecoinvent', 'location': 'RER', 'unit': 'megajoule', 'type': 'technosphere', 'reference product': 'heat, from steam, in chemical industry'}
{'name': 'market for acrylonitrile', 'amount': 1.1200564971751414, 'database': 'ecoinvent', 'location': 'GLO', 'unit': 'kilogram', 'type': 'technosphere', 'reference product': 'acrylonitrile'}
{'name': 'market group for electricity, low voltage', 'amount': 2.5, 'database': 'ecoinvent', 'location': 'RER', 'unit': 'kilowatt hour', 'type': 'technosphere', 'reference product': 'electricity, low voltage'}
{'name': 'market for methyl acrylate', 'amount': 0.05, 'database': 'ecoinvent', 'location': 'GLO', 'unit': 'kilogram', 'type': 'technosphere', 'reference product': 'methyl acrylate'}
{'name': 'market for acrylic acid', 'amount': 0.01, 'database': 'ecoinvent', 'location': 'RER', 'unit': 'kilogram', 'type': 'technosphere', 'reference product': 'acrylic 

OK, some unlinked exchanges are clearly from ecoinvent. Let's try to link those.

In [8]:
imp.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.statistics()

Applying strategy: link_iterable_by_fields
Graph statistics for `carbon fiber` importer:
10 graph nodes:
	None: 10
109 graph edges:
	technosphere: 90
	production: 10
	biosphere: 9
95 edges to the following databases:
	ecoinvent-3.10-cutoff: 68
	carbon fiber: 19
	ecoinvent-3.10-biosphere: 8
3 unique unlinked edges (14 total):
	technosphere: 2
	biosphere: 1




(10, 109, 14, 0)

Let's check the unlinked edges:

In [10]:
for edge in imp.unlinked:
    print(edge)

{'name': 'market for ethylene glycol', 'amount': 2.4225526641883498e-06, 'database': 'ecoinvent', 'location': 'GLO', 'unit': 'kilogram', 'type': 'technosphere', 'reference product': 'ethylene glycol'}
{'name': 'air separation, cryogenic', 'amount': 0.005396530359355638, 'database': 'ecoinvent', 'location': 'RER', 'unit': 'kilogram', 'type': 'technosphere', 'reference product': 'nitrogen, liquid'}
{'name': 'Argon-40', 'amount': 0.012598425196850395, 'unit': 'kilogram', 'categories': ('air',), 'type': 'biosphere', 'uncertainty type': 2, 'loc': -4.3741834572128555, 'u1': 1.5, 'u2': 1.2, 'u3': 1.5, 'u4': 1.1, 'u5': 2, 'u6': 1.2, 'ub': 1.5, 'scale': 0.5121584730617011}


Let's look at the missing Argon flow first. Let's check our database layout:

In [11]:
bd.databases

Databases dictionary with 3 object(s):
	bike
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

That should be in the `ecoinvent-3.10-biosphere` database. Let's search for argon:

In [16]:
[x for x in bd.Database("ecoinvent-3.10-biosphere") if "argon" in x["name"].lower()]

['Argon' (kilogram, None, ('natural resource', 'in air')),
 'Argon-41' (kilo Becquerel, None, ('air', 'low population density, long-term')),
 'Argon-41' (kilo Becquerel, None, ('air', 'urban air close to ground')),
 'Argon-41' (kilo Becquerel, None, ('air',)),
 'Argon-41' (kilo Becquerel, None, ('air', 'non-urban air or from high stacks')),
 'Argon' (kilogram, None, ('air',))]

OK, so we have the following. In our imported data:

{'name': 'Argon-40', 'amount': 0.012598425196850395, 'unit': 'kilogram', 'categories': ('air',), 'type': 'biosphere', 'uncertainty type': 2, 'loc': -4.3741834572128555, 'u1': 1.5, 'u2': 1.2, 'u3': 1.5, 'u4': 1.1, 'u5': 2, 'u6': 1.2, 'ub': 1.5, 'scale': 0.5121584730617011}

In [18]:
import randonneur_data as rd
registry = rd.Registry()
list(registry)

['simapro-ecoinvent-3.10-cutoff',
 'simapro-ecoinvent-3.8-cutoff',
 'simapro-ecoinvent-3.9.1-cutoff',
 'ecoinvent-3.7.1-cutoff-ecoinvent-3.8-cutoff',
 'ecoinvent-3.8-cutoff-ecoinvent-3.9-cutoff',
 'ecoinvent-3.9.1-cutoff-ecoinvent-3.10-cutoff',
 'ecoinvent-3.8-biosphere-ecoinvent-3.9-biosphere',
 'ecoinvent-3.9.1-biosphere-ecoinvent-3.10-biosphere',
 'generic-brightway-units-normalization',
 'generic-brightway-unit-conversions',
 'simapro-9-ecoinvent-3-context',
 'SimaPro-9-ecoinvent-3.9-biosphere',
 'ecoinvent-2.2-biosphere-context-ecoinvent-3.0-biosphere-context',
 'ecoinvent-2.2-biosphere-ecoinvent-3.0-biosphere',
 'simapro-ecoinvent-3.5-apos',
 'simapro-ecoinvent-3.5-cutoff',
 'simapro-ecoinvent-3.5-consequential',
 'SimaPro-9-ecoinvent-3.8-biosphere-manual-matches',
 'SimaPro-9-ecoinvent-3.9-biosphere-manual-matches',
 'simapro-9-ecoinvent-3-water-slash-m3',
 'agribalyse-3.1.1-delete-aggregated-ecoinvent',
 'agribalyse-3.1.1-restore-simapro-ecoinvent-names',
 'agribalyse-3.1.1-bio

In [19]:
registry.sample('ecoinvent-3.9.1-biosphere-ecoinvent-3.10-biosphere')

{'replace': [{'source': {'name': 'Ethyne',
    'formula': 'C2H2',
    'unit': 'kg',
    'uuid': '42a2f3e7-5393-4a69-9b0b-f70a9139ee24'},
   'target': {'name': 'Acetylene',
    'uuid': '42a2f3e7-5393-4a69-9b0b-f70a9139ee24'},
   'comment': 'Flow attribute change not listed in change report'},
  {'source': {'name': 'Propylamine',
    'formula': 'C3H9N',
    'unit': 'kg',
    'uuid': '813b3b1d-4b12-44cd-bbb9-5c384ba019d2'},
   'target': {'name': 'Propyl amine',
    'uuid': '813b3b1d-4b12-44cd-bbb9-5c384ba019d2'},
   'comment': 'Flow attribute change not listed in change report'}],
 'delete': [{'source': {'uuid': '91861063-1826-4860-9957-7c5bde5817a6',
    'name': 'Salt water (obsolete)'},
   'comment': 'There is no salt water flow in ecoinvent.'}]}

In [20]:
imp.randonneur?

Signature:
imp.randonneur(
    label: Optional[str] = None,
    data_registry_path: Optional[pathlib.Path] = None,
    datapackage: Optional[randonneur.datapackage.Datapackage] = None,
    fields: Optional[list] = None,
    mapping: Optional[dict] = None,
    node_filter: Optional[Callable] = None,
    edge_filter: Optional[Callable] = None,
    verbose: bool = False,
    case_sensitive: bool = False,
    add_extra_attributes: bool = True,
    verbs: Optional[List[str]] = ['update', 'replace', 'disaggregate'],
    migrate_edges: bool = True,
    migrate_nodes: bool = False,
) -> None
Docstring:
Apply a stored transformation from `randonneur_data`. See the `randonneur`
[README](https://github.com/brightway-lca/randonneur/blob/main/README.md) and the
[current registry](https://github.com/brightway-lca/randonneur_data/blob/main/randonneur_data/data/registry.json)
for more information.

`label`: Label for the transformation if loading from the `randonneur_data` registry.

`data_registry_pa

In [8]:
imp.strategies

[<function bw2io.strategies.csv.csv_restore_tuples(data)>,
 <function bw2io.strategies.csv.csv_restore_booleans(data)>,
 <function bw2io.strategies.csv.csv_numerize(data)>,
 <function bw2io.strategies.csv.csv_drop_unknown(data)>,
 <function bw2io.strategies.csv.csv_add_missing_exchanges_section(data)>,
 <function bw2io.strategies.generic.normalize_units(db)>,
 <function bw2io.strategies.biosphere.normalize_biosphere_categories(db, lcia=False)>,
 <function bw2io.strategies.biosphere.normalize_biosphere_names(db, lcia=False)>,
 <function bw2io.strategies.biosphere.strip_biosphere_exc_locations(db)>,
 <function bw2io.strategies.generic.set_code_by_activity_hash(db, overwrite=False)>,
 functools.partial(<function link_iterable_by_fields at 0x7f21e27f0dc0>, other=Brightway2 SQLiteBackend: ecoinvent-3.10-biosphere, kind='biosphere'),
 <function bw2io.strategies.generic.assign_only_product_as_production(db)>,
 <function bw2io.strategies.generic.link_technosphere_by_activity_hash(db, external_

It's not going to hurt anything to apply the default transformations, and that will apply the linking functions internally.

In [1]:
import randonneur_data

In [ ]:
bd.projects.set_current("<project name>")

In [2]:
bd.projects.set_current("ecoinvent-3.10-cutoff")

In [ ]:
bi.create_core_migrations()

In [ ]:
xl_importer = bi.ExcelImporter("lci-bike.xlsx")

In [ ]:
xl_importer.apply_strategies()

In [ ]:
xl_importer.statistics()

In [ ]:
for obj in xl_importer.unlinked:
    print(obj)

In [ ]:
xl_importer.match_database(fields=['name'])
xl_importer.statistics()

In [ ]:
xl_importer.write_database()

In [ ]:
bd.databases